In [1]:
import os
import re
import sys
import time
import nltk
import openpyxl
import collections
import numpy as np
import pandas as pd
import pandas.io.formats.excel
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import label_binarize
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def read_data(path):
    data = pd.read_excel(path)
    return data

In [3]:
def get_service_name(data):
    service_name = pd.DataFrame(data['SERVICE_NAME'])
    return service_name

In [4]:
def get_data_by_service_type(data, types):
    data = data[data['SERVICE_TYPE'].isin(types)]
    data.reset_index(inplace=True,drop=True)
    return data

In [5]:
def get_standards_path():
    paths = []
    for file in os.listdir("Standards"):
        paths.append(os.path.join("Standards", file))
    return paths

In [6]:
def read_standards(paths):
    stds = []
    for path in paths:
        stds.append(pd.read_excel(path))
    return stds

In [7]:
def get_providers_path():
    paths = []
    for file in os.listdir("Providers"):
        paths.append(os.path.join("Providers", file))
    return paths

In [8]:
def create_document(DF1,col_index): #DF1 is the DF to be preprocessed #col_index is the number of the describtion column (int)
    # Making A copy to prevent change to the original DF as it is needd subsequently
    DF = DF1.copy()
    # Insuring string type for the wanted col
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].astype(str)
    # Removing all newline and replacing it with space
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].str.replace('\n',' ')
    #Replacing all non alphanumeric in both english and arabic by space
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: re.sub('[^0-9a-zA-Z\u0627-\u064a]+', ' ', x))
    #Separating camel case with space 
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: re.sub(r"([a-z])([A-Z])", r"\1 \2", x))
    #lowercasing all describtions
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join(x.lower() for x in x.split()))
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x:''.join(i for i in x if not i.isdigit()))
    #In case a med dict present (referring the medical abbreviation to its origin) this line return the abbreviation to origin
    ## DF_document.document = DF_document.document.apply(lambda x: " ".join(med_dict(word) for word in x.split()) )
    #Counting all Words that occur either in very low freq or very high and removing them (Threshhold must be configured)
    ## freq = pd.Series(' '.join(DF[DF.columns[col_index]]).split()).value_counts()[:50]
    ## DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join(x for x in x.split() if  x not in freq.index))
    ## freq = pd.Series(' '.join(DF[DF.columns[col_index]]).split()).value_counts()[-1034:]
    ## DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join(x for x in x.split() if x not in freq.index))
    #Defining a stemmer and stemming all words in the description
    st = PorterStemmer()
    #DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: "" if x.isdigit() == True else x)
    return DF

In [9]:
def concat(service_name,standards):
    DF = pd.concat(objs = [service_name,standards])
    DF = DF.apply(lambda x: ''.join(i for i in x if not i.isdigit()))
    return DF

In [10]:
def create_vectorizer(df,service_name,standard):
    stopWords = stopwords.words('english')
    tfidf = TfidfVectorizer(stop_words=stopWords)
    tfidf.fit(df)
    tfref = tfidf.transform(standard)
    tfdata = tfidf.transform(service_name)
    return tfref,tfdata

In [11]:
def find_similar(tfidf_data,tfidf_ref, index, rank = 0):
    #multiplying the 2 matrix to find the cosine similarity
    cosine_similarities = linear_kernel(tfidf_data[index:index+1], tfidf_ref[:]).flatten()
    #getting the most similar index
    highest_ind = cosine_similarities.argsort()[::-1][rank]
    #return the index along with its similarity
    return highest_ind, cosine_similarities[highest_ind]

In [12]:
def get_similarity(data,tfdata,tfref):
    result = pd.DataFrame(columns = ['Serv','cpt','Sim','Num'])
    resultAll = pd.DataFrame(columns = ['Serv','cpt','Sim','Num'])
    for i in range(len(data)):
        for j in range(0,1):
            index, simlarity = find_similar(tfdata, tfref, i, j)
            result.loc[i] = [i, index, simlarity, j]
            resultAll = resultAll.append(result)
            result.drop(result.index, inplace = True)
    return resultAll

In [13]:
def get_max_results(result_ACHI, result_SFDA, result_CPT):
    df = pd.DataFrame(columns=['Serv','cpt','Sim','Num','Mapped_Code'])
    for i in range(len(result_ACHI)):
        sims=[result_ACHI.iloc[i,2], result_SFDA.iloc[i,2], result_CPT.iloc[i,2]]
        if np.argmax(np.array(sims)) == 0:
            df.loc[i] = result_ACHI.iloc[i].to_list() + ['ACHI']
        elif np.argmax(np.array(sims)) == 1:
            df.loc[i] = result_SFDA.iloc[i].to_list() + ['SFDA']
        elif np.argmax(np.array(sims)) == 2:
            df.loc[i] = result_CPT.iloc[i].to_list() + ['CPT']
    return df

In [14]:
def do_map(data,result,ACHI,SFDA,CPT):
    data['Mapped_Code'] = result['Mapped_Code']
    for i in range(len(data)):
        if result.iloc[i,4] == 'ACHI':
            data.loc[i, 'Unified_Code'] = ACHI.iloc[int(result.iloc[i,1]),0]
            data.loc[i, 'Unified_Code_Description'] = ACHI.iloc[int(result.iloc[i,1]),3]
        elif result.iloc[i,4] == 'SFDA':
            data.loc[i, 'Unified_Code'] = SFDA.iloc[int(result.iloc[i,1]),0]
            data.loc[i, 'Unified_Code_Description'] = SFDA.iloc[int(result.iloc[i,1]),3]
        elif result.iloc[i,4] == 'CPT':
            data.loc[i, 'Unified_Code'] = CPT.iloc[int(result.iloc[i,1]),0]
            data.loc[i, 'Unified_Code_Description'] = CPT.iloc[int(result.iloc[i,1]),1]
            
    data['Similarity'] = result['Sim']
    return data

In [15]:
def filter_by_similarity(data, thresh):
    data.loc[data['Similarity'] < thresh, ['Unified_Code','Unified_Code_Description','Mapped_Code']] = 'Not Found!'
    return data

In [16]:
def files_mapping(data_path):
    data = read_data(data_path)
    data = get_data_by_service_type(data, ['Lab','Diagnosis Procedure','Dental','Diagnostic Procedures','Other Medical Services','Package Deal','Physiotherapy','Radiology'])
    ACHI, ACHI_3ADDA, CPT, SFDA = read_standards(get_standards_path())
    service_name = get_service_name(data)
    
    ACHI_processed = create_document(ACHI,3)
    SFDA_processed = create_document(SFDA,3)
    CPT_processed = create_document(CPT,1)
    data_processed = create_document(service_name,0)
    
    if len(data_processed) > 0:
        tfref_ACHI,tfdata_ACHI = create_vectorizer(concat(data_processed['SERVICE_NAME'],ACHI_processed['ascii_desc']),data_processed['SERVICE_NAME'],ACHI_processed['ascii_desc'])
        tfref_SFDA,tfdata_SFDA = create_vectorizer(concat(data_processed['SERVICE_NAME'],SFDA_processed['intended_purpose']),data_processed['SERVICE_NAME'],SFDA_processed['intended_purpose'])
        tfref_CPT,tfdata_CPT = create_vectorizer(concat(data_processed['SERVICE_NAME'],CPT_processed['LongDescription']),data_processed['SERVICE_NAME'],CPT_processed['LongDescription'])

        resultAll_ACHI = get_similarity(data,tfdata_ACHI,tfref_ACHI)
        resultAll_SFDA = get_similarity(data,tfdata_SFDA,tfref_SFDA)
        resultAll_CPT = get_similarity(data,tfdata_CPT,tfref_CPT)

        AllResult = get_max_results(resultAll_ACHI,resultAll_SFDA,resultAll_CPT)
        AllResult = do_map(data,AllResult,ACHI,SFDA,CPT)
        #AllResult = filter_by_similarity(AllResult, 0.6)
    else:
        AllResult = []

    return AllResult

In [17]:
def export_excel(path):
    writer = pd.ExcelWriter(path,engine='openpyxl')
    #writer.book = openpyxl.load_workbook(path).sheetnames
    
    pandas.io.formats.excel.ExcelFormatter.header_style = None
    
    result.to_excel(writer,sheet_name=openpyxl.load_workbook(path).sheetnames[0],index=False)
    
    for i, sheetname in enumerate(writer.book.sheetnames):
        worksheet = writer.book[sheetname]
        mediumStyle = openpyxl.worksheet.table.TableStyleInfo(name='TableStyleMedium2',showRowStripes=True)
        table = openpyxl.worksheet.table.Table(ref=worksheet.dimensions,displayName="Table" + str(i),tableStyleInfo=mediumStyle)
        worksheet.add_table(table)
        
        for col in worksheet.columns:
            worksheet.column_dimensions[col[0].column_letter].width = (len(str(col[0].value)) + 2) * 1.2
    
    writer.book.save(path)
    writer.book.close()
    writer.close()
    os.rename(path,os.path.join("Mapped Providers 2",os.path.splitext(os.path.basename(path))[0]+" Mapped"+os.path.splitext(path)[-1]))

In [18]:
%%time
n = 1
for provider_path in get_providers_path()[23:]:
    start_time = time.time()
    print("Started mapping " + os.path.basename(provider_path))
    result = files_mapping(provider_path)
    if len(result) > 0:
        print("Finished mapping " + os.path.basename(provider_path))
        export_excel(provider_path)
        print("Exported " + os.path.basename(provider_path) + " to the 'Mapped Providers 2' folder")
        elapsed_time = time.time() - start_time
        print("Time: " + str(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))) + "\n")
    else:
        print("There is no mappings for this provider\n")

Started mapping Hayat National Hospital - Qassim ## 6212000853 @ 3603.xlsx
Finished mapping Hayat National Hospital - Qassim ## 6212000853 @ 3603.xlsx
Exported Hayat National Hospital - Qassim ## 6212000853 @ 3603.xlsx to the 'Mapped Providers 2' folder
Time: 00:03:42

Started mapping Hiba Asia Polyclinic ## 6212000646 @ 6949.xlsx
Finished mapping Hiba Asia Polyclinic ## 6212000646 @ 6949.xlsx
Exported Hiba Asia Polyclinic ## 6212000646 @ 6949.xlsx to the 'Mapped Providers 2' folder
Time: 00:01:21

Started mapping Hiba Asia Polyclinic 2 Jeddah ## 6212000690 @ 6949.xlsx
Finished mapping Hiba Asia Polyclinic 2 Jeddah ## 6212000690 @ 6949.xlsx
Exported Hiba Asia Polyclinic 2 Jeddah ## 6212000690 @ 6949.xlsx to the 'Mapped Providers 2' folder
Time: 00:00:59

Started mapping Jeddah National Hospital ## 6212000297 @ 1652.xlsx
Finished mapping Jeddah National Hospital ## 6212000297 @ 1652.xlsx
Exported Jeddah National Hospital ## 6212000297 @ 1652.xlsx to the 'Mapped Providers 2' folder
Time: